In [12]:
import dao.MongoTagUser as mtu
import panda.TagUser as tupd

mongotu = mtu.TagofSongUserRecom(user_id="0bc2cc42a7be11edb7f600155daffd24")
# list = mongotu.makeRecomUsersSet()
# print(list)

In [13]:
doc = mongotu.rdb.findDocument(
            collection_name="like", query=mongotu.query, projection=mongotu.projections)
# 储存请求用户的名字
mongotu.user_name = doc[0]['name']
# 随机获取一定数量的用户来计算tag 相似度
# 统一获取其他用户的信息
querys = [{'$sample': {"size": 50-1}}]
project = {'$project': mongotu.a_projections}
querys.append(project)
docs = mongotu.rdb.aggregateDocument(
     collection_name="like", querys=querys)
print(doc)

[{'id': '0bc2cc42a7be11edb7f600155daffd24', 'name': 'RuRuUIH', 'tags': ['蓝调', '感动', '民族', '夜晚', 'New Age', '华语', '安静', '英伦', '酒吧', '浪漫', '快乐', '拉丁', '粤语', '90后', '古风', '朋克', '兴奋', '流行', '怀旧', '游戏', '经典', '运动', '爵士', 'KTV', '驾车', '小语种', '吉他', '雷鬼', '乡村', '舞曲', '说唱', '摇滚', '金属', '伤感', '翻唱', 'ACG', '影视原声', '放松', '治愈', '轻音乐', '工作', '电子', '日语', '下午茶', '韩语', '学习', '旅行', '器乐', '清新', '孤独', 'Bossa Nova', '欧美', 'R&B/Soul', '后摇', '性感', '古典', '钢琴', '民谣', '另类/独立']}]


In [14]:
# 判断 docs中是否存在 doc 并合并
docs = mongotu.isDocinDocs(doc=doc, docs=docs)
# print(docs)     

In [15]:
# 查找数据songs 便于 统计tags 出现的次数
# 提取docs 中用户的id，之后用
list_name = mongotu.getItemFromDocs(docs=docs, key='name')
list_id = mongotu.getItemFromDocs(docs=docs, key='id')
# 通过这个方法获取songs，来计算tag出现次数
docs_song = mongotu.getSongsforTagfromMongo(list_id)
# count_tag 统计
dict_tags = mongotu.makeMatricTagCountData(docs=docs_song)
# print(dict_tags)
        

In [16]:
# 根据以上docs 来生成 xy轴，矩阵
# 内容
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongotu.makeXYMatric(docs=docs)
# print(data)
        

  (34, 0)	1
  (5, 0)	1
  (44, 0)	1
  (61, 0)	1
  (26, 0)	1
  (57, 0)	1
  (22, 0)	1
  (51, 0)	1
  (18, 0)	1
  (20, 0)	1
  (16, 0)	1
  (41, 0)	1
  (54, 0)	1
  (62, 0)	1
  (37, 0)	1
  (48, 0)	1
  (52, 0)	1
  (38, 0)	1
  (60, 0)	1
  (50, 0)	1
  (24, 0)	1
  (10, 0)	1
  (0, 0)	1
  (39, 0)	1
  (35, 0)	1
  :	:
  (34, 49)	1
  (45, 49)	1
  (14, 49)	1
  (53, 49)	1
  (20, 49)	1
  (51, 49)	1
  (24, 49)	1
  (63, 49)	1
  (37, 49)	1
  (33, 49)	1
  (3, 49)	1
  (10, 49)	1
  (52, 49)	1
  (1, 49)	1
  (12, 49)	1
  (11, 49)	1
  (19, 49)	1
  (5, 49)	1
  (48, 49)	1
  (16, 49)	1
  (58, 49)	1
  (15, 49)	1
  (9, 49)	1
  (62, 49)	1
  (22, 49)	1


In [19]:
# 生成pandas 表格
tudf = tupd.TUPandas(datas=list(
map(list, data.toarray())), items=mongotu.y, users=mongotu.x)
# print(tudf.df)     


In [21]:
# 开始将tudf 中的数据根据上面的dict_tags中的数据进行替换
tudf = mongotu.changeDataCountTagTUDF(list_name, dict_tags, tudf)
# print(tudf.df)        

In [26]:
# 推荐过程
similar_user = tudf.makeSimilarityWithPearson()

In [27]:
similar_user

,RuRuUIH,De3Y7,B9pB5jlUI,6OB8MvSZ,Kb,Os0T,3IeaCc,5h0kcmSoUP,OXmegpgti,KN,...,mfYL3,hXvmT,n2nP56u6v,rzQlciv,x0,2CQ2,K0Kcll3Ca,f65TKu,K,WMq0N
RuRuUIH,1.00,0.96,0.92,0.97,0.96,0.08,0.95,0.14,0.97,0.98,...,0.97,0.94,0.96,0.95,0.90,0.95,0.95,0.95,0.97,0.91
De3Y7,0.96,1.00,0.97,0.98,0.97,0.07,0.97,0.11,0.98,0.99,...,0.98,0.97,0.97,0.96,0.94,0.97,0.96,0.96,0.98,0.88
B9pB5jlUI,0.92,0.97,1.00,0.95,0.91,0.09,0.92,0.07,0.94,0.95,...,0.94,0.96,0.94,0.93,0.92,0.95,0.94,0.93,0.97,0.85
6OB8MvSZ,0.97,0.98,0.95,1.00,0.97,0.10,0.96,0.13,0.98,0.98,...,0.97,0.97,0.97,0.95,0.91,0.96,0.96,0.96,0.98,0.90
Kb,0.96,0.97,0.91,0.97,1.00,0.01,0.96,0.16,0.99,0.97,...,0.97,0.95,0.97,0.96,0.93,0.95,0.95,0.94,0.96,0.89
Os0T,0.08,0.07,0.09,0.10,0.01,1.00,0.02,0.04,0.03,0.08,...,0.04,0.02,0.04,-0.01,-0.04,-0.01,0.08,0.06,0.06,0.09
3IeaCc,0.95,0.97,0.92,0.96,0.96,0.02,1.00,0.08,0.97,0.97,...,0.97,0.95,0.96,0.94,0.92,0.96,0.94,0.94,0.96,0.89
5h0kcmSoUP,0.14,0.11,0.07,0.13,0.16,0.04,0.08,1.00,0.10,0.10,...,0.10,0.16,0.19,0.15,-0.01,0.11,0.17,0.17,0.10,0.16
OXmegpgti,0.97,0.98,0.94,0.98,0.99,0.03,0.97,0.10,1.00,0.99,...,0.98,0.96,0.98,0.97,0.93,0.96,0.96,0.95,0.97,0.88
KN,0.98,0.99,0.95,0.98,0.97,0.08,0.97,0.10,0.99,1.00,...,0.98,0.97,0.97,0.96,0.92,0.96,0.97,0.96,0.98,0.89


In [28]:
# 获取相似度最高几位用户sign 设置为 1 表示 使用Pearson相关系数获取相关性最高的几个用户
diction = tudf.makeTopNUsers(similar=similar_user,sign=1)
print(diction)

{'RuRuUIH': ['9w6CypKgN', 'jXS0nZ', 'KN', 'MN', 'pQ2n0M', 't9', 'KAxYKnia', 'PHatsL', 'z8XrbfOL6', 'DM7IN5IY', 'xCi', '9qqrh', 'OXmegpgti', 'hrgHH5Y8s', 'K', 'mfYL3', '6OB8MvSZ', 'VHA', 'oqs8rJXaag', 'AVLzWuOnm', 'c7', 'Fd', 'pAiVAPa', 'De3Y7', 'Kb', 'npwW8p7B', 'tuk6Fe', 'VQnw', 'n2nP56u6v', '04']}


In [ ]:
# 利用当前用户的信息确定他们的id值形成对应的list_id
diction[mongotu.user_name] = mongotu.makeListIdWithUserName(diction[mongotu.user_name])

In [ ]:
# 保存过程
# mongotu.saveDataSetRecomAnswer(diction)
# 改变形式，方便存储
docs = mongotu.changeDataFormat(diction=diction, list_keys=list(diction))
# 保存
mongotu.wdb.writeDocument(docs=docs, collection_name="recom")